# Day 7: Some Assembly Required

[*Advent of Code 2015 day 7*](https://adventofcode.com/2015/day/7) and [*solution megathread*](https://redd.it/3vr4m4)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/07/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F07%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


In [2]:
HTML(downloaded['part1'])

In [3]:
testdata = (['123 -> x',
             '456 -> y',
             'x AND y -> d',
             'x OR y -> e',
             'x LSHIFT 2 -> f',
             'y RSHIFT 2 -> g',
             'NOT x -> h',
             'NOT y -> i'],
            {'d': 72,
             'e': 507,
             'f': 492,
             'g': 114,
             'h': 65412,
             'i': 65079,
             'x': 123,
             'y': 456})

inputdata = downloaded['input'].splitlines()

In [4]:
inputdata[1:5]

['kg OR kf -> kh', 'ep OR eo -> eq', '44430 -> b', 'NOT gs -> gt']

In [5]:
# def my_part1_solution(data):
#     ml = MillionLights()
#     return(ml.execute_instructions(list(map(LightInstruction, data))))

In [6]:
HTML(downloaded['part1_footer'])

In [7]:
# HTML(downloaded['part2'])

In [8]:
# starttime = timeit.default_timer()
# ml = MillionLights2()
# for instruction in inputdata:
#    ml.execute_instruction(LightInstruction2(instruction))
# print(ml.count_lit())
# print("The time difference is :", timeit.default_timer() - starttime)

In [9]:
# HTML(downloaded['part2_footer'])